In [1]:
%load_ext autoreload
%autoreload 2

import os

os.chdir("..")
xla_flags = os.environ.get("XLA_FLAGS", "")
xla_flags += " --xla_gpu_triton_gemm_any=True"
os.environ["XLA_FLAGS"] = xla_flags
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["MUJOCO_GL"] = "egl"
os.environ["PYOPENGL_PLATFORM"] = "egl"

import functools
import json
from datetime import datetime

import jax
from jax.experimental import checkify
import jax.numpy as jp
import matplotlib.pyplot as plt
import mediapy as media
import mujoco
import wandb
from brax.training.agents.ppo import networks as ppo_networks
from brax.training.agents.ppo import train as ppo
from etils import epath
from flax.training import orbax_utils
from IPython.display import clear_output, display
from orbax import checkpoint as ocp
from tqdm import tqdm

from mujoco_playground import locomotion, wrapper
from mujoco_playground.config import locomotion_params

from vnl_mjx.tasks.mouse import mouse_reach, mouse_reach_josh

jax.config.update("jax_debug_nans", True)

# # Enable persistent compilation cache.
jax.config.update("jax_compilation_cache_dir", "/tmp/jax_cache")
jax.config.update("jax_persistent_cache_min_entry_size_bytes", -1)
jax.config.update("jax_persistent_cache_min_compile_time_secs", 0)

In [2]:
from mujoco_playground import wrapper as mp_wrapper
import jax.numpy as jnp
from vnl_mjx.tasks.rodent import flat_arena, bowl_escape
import hydra
from omegaconf import DictConfig, OmegaConf
from track_mjx.agent import checkpointing

In [3]:
# idk stuff scott defined
local_devices_to_use = jax.local_device_count()
key = jax.random.PRNGKey(0)
global_key, local_key = jax.random.split(key)
del key
process_count = jax.process_count()
process_id = jax.process_index()
local_key = jax.random.fold_in(local_key, process_id)
local_key, key_env, eval_key = jax.random.split(local_key, 3)
num_envs = 1

In [6]:
env = mouse_reach_josh.MouseEnv()
# env.add_target(pos=jp.array([0.004, 0.012, -0.006]))
# env = flat_arena.FlatWalk(config_overrides=env_config)
# env.target
# env._spec.worldbody.add_site(
#             name="target",
#             pos=env.target,
#             size=[0.001, 0.001, 0.001],
#             rgba=[0, 1, 0, 0.5],
#         )
env.reset(0)

UPDATED HAND MOUSE
target added to postion;  [0.00965321 0.00314682 0.0063303 ]
Env reset with target position [0.00965321 0.00314682 0.0063303 ] with type <class 'jaxlib.xla_extension.ArrayImpl'>


State(data=Data(ne=np.int64(0), nf=np.int64(0), nl=np.int64(4), nefc=np.int64(4), ncon=0, solver_niter=Array(0, dtype=int32), time=Array(0., dtype=float32), qpos=Array([0., 0., 0., 0.], dtype=float32), qvel=Array([0., 0., 0., 0.], dtype=float32), act=Array([], shape=(0,), dtype=float32), qacc_warmstart=Array([ 8.8769996e-01,  1.0206248e-03,  2.9429259e+00, -7.1665111e+00],      dtype=float32), ctrl=Array([0., 0., 0., 0.], dtype=float32), qfrc_applied=Array([0., 0., 0., 0.], dtype=float32), xfrc_applied=Array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]], dtype=float32), eq_active=Array([], shape=(0,), dtype=uint8), mocap_pos=Array([], shape=(0, 3), dtype=float32), mocap_quat=Array([], shape=(0, 4), dtype=float32), qacc=Array([ 8.8769996e-01,  1.

In [25]:
# type(env._mj_model)
# m = env._mj_model
print([m.geom(i).name for i in range(m.ngeom)])
print()
print(m.geom(7))

['clavicle_mesh', 'scapula_mesh', 'humerus_mesh', 'ulna_mesh', 'elbow_marker', 'radius_mesh', 'wrist_marker', 'hand_mesh_geom']

<_MjModelGeomViews
  bodyid: array([8], dtype=int32)
  conaffinity: array([0], dtype=int32)
  condim: array([3], dtype=int32)
  contype: array([1], dtype=int32)
  dataid: array([5], dtype=int32)
  friction: array([1.e+00, 5.e-03, 1.e-04])
  gap: array([0.])
  group: array([0], dtype=int32)
  id: 7
  margin: array([0.])
  matid: array([-1], dtype=int32)
  name: 'hand_mesh_geom'
  pos: array([0.000172  , 0.0022888 , 0.00101112])
  priority: array([0], dtype=int32)
  quat: array([1., 0., 0., 0.])
  rbound: array([0.00270284])
  rgba: array([0.5, 0.5, 0.5, 1. ], dtype=float32)
  sameframe: array([3], dtype=uint8)
  size: array([0.0010469 , 0.00218949, 0.00118976])
  solimp: array([9.0e-01, 9.5e-01, 1.0e-03, 5.0e-01, 2.0e+00])
  solmix: array([1.])
  solref: array([0.02, 1.  ])
  type: array([7], dtype=int32)
  user: array([], dtype=float64)
>


In [7]:
wrap_for_training = mp_wrapper.wrap_for_brax_training
wrap_env = wrap_for_training(
        env,
        episode_length=10,
        action_repeat=1,
        randomization_fn=None,
        # use_lstm=use_lstm,
    )

reset_fn = jax.jit(jax.vmap(wrap_env.reset))
key_envs = jax.random.split(key_env, num_envs // process_count)
key_envs = jnp.reshape(key_envs, (local_devices_to_use, -1) + key_envs.shape[1:])
env_state = reset_fn(key_envs)

Env reset with target position [0.00965321 0.00314682 0.0063303 ] with type <class 'jaxlib.xla_extension.ArrayImpl'>


In [8]:
for a in env_state.info.keys():
    print(a)
    

episode_done
episode_metrics
first_obs
first_state
last_action
proprioceptive_obs_size
reference_obs_size
steps
target_position
truncation


In [11]:
env_state.info['last_action']

Array([[[0., 0., 0., 0.]]], dtype=float32)

In [16]:
from brax.training.agents.ppo import networks as ppo_networks

In [19]:
env_state.obs.shape

(1, 1, 11)